In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import display, Markdown
import plotly.express as px
import os

In [2]:
CSV_PATH = Path("/Users/rushil/brain_extraction/results/quantitative/pairwise_comparison/pairwise_2x2_metrics_all_scans.csv")
df = pd.read_csv(CSV_PATH)
display(df.head())

exclude = ['6109-317_20150302_0647_ct', '6142-308_20150610_0707_ct', '6193-324_20150924_1431_ct', '6257-335_20160118_1150_ct',
                     '6418-193_20161228_1248_ct', '6470-296_20170602_0607_ct', '6480-154_20170622_0937_ct']

exclude_prefixes = ("6046", "6084", "6096", "6246", "6315", "6342", "6499")

df = df[~df["stem"].isin(exclude)]
df = df[~df["stem"].str.startswith(exclude_prefixes)]

,patient_id,stem,method_A,method_B,tp,fp,fn,tn,dice,iou,sensitivity_sym,specificity_sym,msd_mm,hd95_mm,icv_A_ml,icv_B_ml,delta_icv_ml,delta_icv_pct,n_vox
0,6001-161,6001-161_20131229_1702_ct,Brainchop,CTBET,1285727,68597,21734,8761542,0.966064,0.934355,0.966363,0.994879,0.646912,3.936647,1614.304686,1558.445704,55.858982,3.584275,10137600
1,6001-161,6001-161_20131229_1702_ct,Brainchop,CTbet_Docker,1288829,65495,18445,8764831,0.968463,0.938854,0.968765,0.995241,0.665632,3.813597,1614.304686,1558.222807,56.081880,3.599092,10137600
2,6001-161,6001-161_20131229_1702_ct,Brainchop,HD-CTBET,1299270,55054,44613,8738663,0.963062,0.928755,0.963076,0.994330,0.999888,4.999462,1614.304686,1601.859396,12.445290,0.776928,10137600
3,6001-161,6001-161_20131229_1702_ct,Brainchop,Robust-CTBET,1284087,70237,15829,8767447,0.967574,0.937185,0.967981,0.995125,0.603386,3.452668,1614.304686,1549.452340,64.852347,4.185501,10137600
4,6001-161,6001-161_20131229_1702_ct,Brainchop,SynthStrip,1340260,14064,64049,8719227,0.971684,0.944928,0.972003,0.995549,0.652497,3.417967,1614.304686,1673.884978,-59.580292,-3.559402,10137600


In [3]:
THRESHOLDS = [0.90, 0.95, 0.97]
outdir = Path("/Users/rushil/brain_extraction/results/quantitative/pairwise_comparison/derived_metrics")
outdir.mkdir(parents=True, exist_ok=True)

# Canonicalize pair order so reversed A/B rows are treated as the same comparison.
df_pairs = df.copy()
df_pairs[['m1', 'm2']] = pd.DataFrame(np.sort(df_pairs[['method_A', 'method_B']].values, axis=1),
                                      index=df_pairs.index)
df_pairs['pair_key'] = df_pairs['stem'].astype(str) + '||' + df_pairs['m1'] + '||' + df_pairs['m2']
df_pairs = df_pairs.drop_duplicates('pair_key').copy()

rows = []
methods = sorted(set(df_pairs['m1']).union(df_pairs['m2']))

# For each method, count each (canonical) comparison row as a separate observation.
for t in THRESHOLDS:
    for m in methods:
        method_df = df_pairs[(df_pairs['m1'] == m) | (df_pairs['m2'] == m)].copy()
        # count comparisons (one row per unordered pair per stem) rather than collapsing by stem
        n_comparisons = int(len(method_df))
        n_ge = int((method_df['dice'] >= t).sum())
        pct = (n_ge / n_comparisons * 100.0) if n_comparisons else np.nan
        rows.append({
            'method': m,
            'threshold': float(t),
            'n_comparisons': n_comparisons,
            'n_ge': n_ge,
            'pct_ge': float(pct)
        })

dice_method_df = pd.DataFrame(rows)
dice_method_df.to_csv(outdir / 'dice_thresholds_by_method_across_others.csv', index=False)

pivot = dice_method_df.pivot(index='method', columns='threshold', values='pct_ge').reindex(index=methods)
pivot.to_csv(outdir / 'dice_matrix_by_method_thresholds.csv')

In [4]:
THRESHOLDS = [2.5, 5.0, 10.0]
outdir = Path("/Users/rushil/brain_extraction/results/quantitative/pairwise_comparison/derived_metrics")
outdir.mkdir(parents=True, exist_ok=True)

# Canonicalize unordered pair (m1,m2) so reversed rows are treated as the same comparison.
df_pairs = df.copy()
df_pairs[['m1', 'm2']] = pd.DataFrame(np.sort(df_pairs[['method_A', 'method_B']].values, axis=1),
                                      index=df_pairs.index)
df_pairs['pair_key'] = df_pairs['stem'].astype(str) + '||' + df_pairs['m1'] + '||' + df_pairs['m2']

# use absolute ICV differences so order (A vs B) sign doesn't matter
df_pairs['abs_delta_icv_ml'] = df_pairs['delta_icv_ml'].abs()

# If both A->B and B->A exist for the same unordered pair, keep the first occurrence (drop duplicates).
# After taking abs delta, both rows should be equivalent; keeping the first is the minimal/fast option.
agg = df_pairs.drop_duplicates('pair_key', keep='first').loc[:, ['pair_key','stem','m1','m2','abs_delta_icv_ml']].copy()

rows = []
methods = sorted(set(df_pairs['m1']).union(df_pairs['m2']))

# For each method, count each canonical unordered-pair observation once and test abs delta against thresholds
for t in THRESHOLDS:
    for m in methods:
        method_df = agg[(agg['m1'] == m) | (agg['m2'] == m)].copy()
        n_comparisons = int(len(method_df))
        n_within = int((method_df['abs_delta_icv_ml'] <= t).sum())
        pct_within = (n_within / n_comparisons * 100.0) if n_comparisons else np.nan
        rows.append({
            'method': m,
            'threshold': float(t),
            'n_comparisons': n_comparisons,
            'n_within': n_within,
            'pct_within': float(pct_within)
        })

icv_within_method_df = pd.DataFrame(rows)
icv_within_method_df.to_csv(outdir / 'icv_within_thresholds_by_method.csv', index=False)

# pivot for reporting / heatmap
pivot_icv = icv_within_method_df.pivot(index='method', columns='threshold', values='pct_within').reindex(index=methods)
pivot_icv.to_csv(outdir / 'icv_within_matrix_by_method.csv')
display(Markdown('### ICV within thresholds by method (first rows)'))
display(icv_within_method_df.head())
display(Markdown('### ICV within matrix (rows=method, cols=threshold)'))
display(pivot_icv)

### ICV within thresholds by method (first rows)

,method,threshold,n_comparisons,n_within,pct_within
0,Brainchop,2.5,27713,841,3.034677
1,CTBET,2.5,27713,2945,10.626782
2,CT_BET,2.5,14418,1488,10.320433
3,CTbet_Docker,2.5,27713,2983,10.763901
4,HD-CTBET,2.5,27713,540,1.948544


### ICV within matrix (rows=method, cols=threshold)

threshold,2.5,5.0,10.0
method,,,
Brainchop,3.034677,6.130697,12.275827
CTBET,10.626782,21.592754,40.403421
CT_BET,10.320433,20.099875,34.352892
CTbet_Docker,10.763901,19.990618,34.590264
HD-CTBET,1.948544,3.915130,7.750875
Robust-CTBET,5.174467,13.845488,35.463501
SynthStrip,0.703641,1.418107,2.908382


In [5]:
# Combined check: dice > 0.97 AND within 10 ml (absolute)
DICE_THRESH = 0.97
ICV_THRESH_ML = 10.0
outdir = Path("/Users/rushil/brain_extraction/results/quantitative/pairwise_comparison/derived_metrics")
outdir.mkdir(parents=True, exist_ok=True)

# Canonicalize unordered pair and compute abs ICV diff
df_pairs = df.copy()
df_pairs[['m1','m2']] = pd.DataFrame(np.sort(df_pairs[['method_A','method_B']].values, axis=1), index=df_pairs.index)
df_pairs['pair_key'] = df_pairs['stem'].astype(str) + '||' + df_pairs['m1'] + '||' + df_pairs['m2']
df_pairs['abs_delta_icv_ml'] = df_pairs['delta_icv_ml'].abs()

# keep a single canonical row per unordered pair (first occurrence)
agg = df_pairs.drop_duplicates('pair_key', keep='first').loc[:, ['pair_key','stem','m1','m2','dice','abs_delta_icv_ml']].copy()

rows = []
methods = sorted(set(df_pairs['m1']).union(df_pairs['m2']))

# For each method, count canonical comparisons and those meeting both criteria
for m in methods:
    method_df = agg[(agg['m1'] == m) | (agg['m2'] == m)].copy()
    n_comp = int(len(method_df))
    n_both = int(((method_df['dice'] > DICE_THRESH) & (method_df['abs_delta_icv_ml'] <= ICV_THRESH_ML)).sum())
    pct_both = (n_both / n_comp * 100.0) if n_comp else np.nan
    rows.append({
        'method': m, 'n_comparisons': n_comp, 'n_both': n_both, 'pct_both': float(pct_both)
    })

combined_df = pd.DataFrame(rows)
combined_df.to_csv(outdir / 'dice_gt0.97_icv_within10_by_method.csv', index=False)
display(Markdown('### Combined: dice > 0.97 AND abs(delta_icv_ml) <= 10 mL (per method)'))
display(combined_df)

# simple vector (percent) for quick reporting / heatmap-ish output
pivot_combined = combined_df.set_index('method')['pct_both'].reindex(index=methods)
pivot_combined.to_csv(outdir / 'dice0.97_icv10_pct_by_method.csv')
display(Markdown('### Percent meeting both criteria (rows=method)'))
display(pivot_combined)

### Combined: dice > 0.97 AND abs(delta_icv_ml) <= 10 mL (per method)

,method,n_comparisons,n_both,pct_both
0,Brainchop,27713,1296,4.676506
1,CTBET,27713,10363,37.394003
2,CT_BET,14418,4565,31.661812
3,CTbet_Docker,27713,8827,31.851478
4,HD-CTBET,27713,200,0.721683
5,Robust-CTBET,27713,9220,33.269585
6,SynthStrip,27713,207,0.746942


### Percent meeting both criteria (rows=method)

method
Brainchop        4.676506
CTBET           37.394003
CT_BET          31.661812
CTbet_Docker    31.851478
HD-CTBET         0.721683
Robust-CTBET    33.269585
SynthStrip       0.746942
Name: pct_both, dtype: float64

In [12]:
outdir = "/Users/rushil/brain_extraction/results/quantitative/pairwise_comparison/derived_metrics"

METRICS = ['dice', 'iou', 'sensitivity_sym', 'msd_mm', 'hd95_mm', 'delta_icv_ml']

# Use mean-only aggregation and render two heatmaps per metric: scan-level and patient-level.
# Canonicalize unordered pairs so A vs B and B vs A are treated once.
for metric in METRICS:
    savedir = os.path.join(outdir, metric)
    os.makedirs(savedir, exist_ok=True)
    
    df_work = df.copy()
    df_work[['m1','m2']] = pd.DataFrame(np.sort(df_work[['method_A','method_B']].values, axis=1), index=df_work.index)

    # for delta_icv_ml we care about absolute value only
    if metric == 'delta_icv_ml':
        df_work['metric_val'] = df_work[metric].abs()
    else:
        df_work['metric_val'] = df_work[metric]

    # scan-level: group by stem and canonical pair, then average (mean)
    scan2scan = df_work.groupby(['stem','m1','m2'], as_index=False)['metric_val'].mean()
    pair_scan = scan2scan.groupby(['m1','m2'], as_index=False)['metric_val'].mean().rename(columns={'metric_val': 'point'})

    methods = sorted(set(list(df_work['m1']) + list(df_work['m2'])))
    mat = pd.DataFrame(np.full((len(methods), len(methods)), np.nan), index=methods, columns=methods, dtype=float)
    for _, r in pair_scan.iterrows():
        a, b, v = r['m1'], r['m2'], r['point']
        ia = methods.index(a)
        ib = methods.index(b)
        # Only fill upper triangle (including a==b if present)
        if ia <= ib:
            mat.iat[ia, ib] = v  # keep full precision here

    # Convert to float matrix and mask lower triangle + diagonal with np.nan so px.imshow leaves them blank
    M = mat.values.astype(float)
    tril_idx = np.tril_indices_from(M, -1)
    M[tril_idx] = np.nan
    np.fill_diagonal(M, np.nan)

    fig = px.imshow(M, x=methods, y=methods, text_auto='.2f', aspect='equal')
    # formatting: black labels, small font, no grid lines or zero lines
    fig.update_xaxes(tickangle=45, tickfont=dict(color='black', size=10), showgrid=False, zeroline=False)
    fig.update_yaxes(tickfont=dict(color='black', size=9), showgrid=False, zeroline=False)
    fig.update_layout(title=f"{metric.upper()} heatmap (scan-level, mean)", xaxis_title='', yaxis_title='', height=700, width=800)
    fig.show()
    fig.write_image(os.path.join(savedir, f"{metric}_scan_level.png"), format="png", scale=4)

    # patient-level: group by patient_id and canonical pair, then average (mean)
    pat2pat = df_work.groupby(['patient_id','m1','m2'], as_index=False)['metric_val'].mean()
    pair_pat = pat2pat.groupby(['m1','m2'], as_index=False)['metric_val'].mean().rename(columns={'metric_val': 'point'})

    mat2 = pd.DataFrame(np.full((len(methods), len(methods)), np.nan), index=methods, columns=methods, dtype=float)
    for _, r in pair_pat.iterrows():
        a, b, v = r['m1'], r['m2'], r['point']
        ia = methods.index(a)
        ib = methods.index(b)
        if ia <= ib:
            mat2.iat[ia, ib] = v

    M2 = mat2.values.astype(float)
    tril_idx2 = np.tril_indices_from(M2, -1)
    M2[tril_idx2] = np.nan
    np.fill_diagonal(M2, np.nan)

    fig2 = px.imshow(M2, x=methods, y=methods, text_auto='.2f', aspect='equal')
    fig2.update_xaxes(tickangle=45, tickfont=dict(color='black', size=10), showgrid=False, zeroline=False)
    fig2.update_yaxes(tickfont=dict(color='black', size=9), showgrid=False, zeroline=False)
    fig2.update_layout(title=f"{metric.upper()} heatmap (patient-level, mean)", xaxis_title='', yaxis_title='', height=700, width=800)
    fig2.show()
    fig2.write_image(os.path.join(savedir, f"{metric}_subject_level.png"), format="png", scale=4)
